In [1]:
import numpy as np
import pandas as pd
import math
import sys
import random
from ipynb.fs.full.Formulas import *
from ipynb.fs.full.Environment import *
from ipynb.fs.full.Flow_Grouping_Algo import *

Bhaichara On Top


## Generating max hop of each flows

In [2]:
max_hops = np.ones(number_of_flows + 1)
for flow_index in range(1,len(flows)):
    random_relay_candidate = flows_candidate_relay_set_pairs[str([flow_index,0.0])]
    if random_relay_candidate[2] != -1:
        max_hops[flow_index] = 2

In [3]:
max_hops

array([1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])

In [2]:
def concurrent_transmission():
    # groups = generate_groups()
    system_throughput = 0
    current_hops = np.ones(number_of_flows + 1)
    number_of_slots = 1
    for group in groups:
        uavs_in_group = []
        bits_in_each_flow = np.zeros(len(group))
        number_of_slots_current_group = 0
        completed_hops = set()
        group_size = len(group)
        for flow_index in group:
            _,relay_candidate_set,random_relay_candidate = flows_candidate_relay_set_pairs[str([flow_index,0.0])]
            for relay_candidate in relay_candidate_set:
                if relay_candidate.type == 'uav':
                    uavs_in_group.append(uav_object_to_index_map[relay_candidate])
        while len(group):
            if completed_hops:
                _,datarate_u2v = calculate_sinr_and_datarate_u2v(group,uavs_in_group,completed_hops)
                _,datarate_v2u = calculate_sinr_and_datarate_v2u(group,uavs_in_group,completed_hops)
                _,datarate_v2v = calculate_sinr_and_datarate_v2v(group,completed_hops)
                for group_index in range(group_size):
                    if bits_in_each_flow[group_index] != -1:
                        bits_in_each_flow[group_index] = number_of_slots_current_group * fun(random_relay_candidate,completed_hops)

                    
                    
            else:
                
    

SyntaxError: incomplete input (634182193.py, line 9)

In [2]:
flows_candidate_relay_set_pairs

{'[1, 0.0]': ['same_lane_relay',
  {<ipynb.fs.full.Environment.UAV at 0x7f60d1cb1d30>,
   <ipynb.fs.full.Environment.Vehicle at 0x7f60d1cb3980>},
 '[2, 0.0]': ['adjacent_lane_relay',
  {<ipynb.fs.full.Environment.UAV at 0x7f60d1cb1d30>,
   <ipynb.fs.full.Environment.Vehicle at 0x7f60d1ce4470>},
 '[3, 0.0]': ['adjacent_lane_relay',
  {<ipynb.fs.full.Environment.UAV at 0x7f60d1cb1d30>,
   <ipynb.fs.full.Environment.Vehicle at 0x7f60d1ce4470>},
 '[4, 0.0]': ['adjacent_lane_relay',
  {<ipynb.fs.full.Environment.UAV at 0x7f60d1cb1d30>,
   <ipynb.fs.full.Environment.Vehicle at 0x7f60d1ce4470>},
 '[5, 0.0]': ['adjacent_lane_relay',
  {<ipynb.fs.full.Environment.UAV at 0x7f60d1cb1d30>,
   <ipynb.fs.full.Environment.Vehicle at 0x7f60d1ce4470>},
 '[6, 0.0]': ['adjacent_lane_relay',
  {<ipynb.fs.full.Environment.UAV at 0x7f60d1cb1d30>,
   <ipynb.fs.full.Environment.Vehicle at 0x7f60d1ce4470>},
 '[7, 0.0]': ['adjacent_lane_relay',
  {<ipynb.fs.full.Environment.UAV at 0x7f60d1cb1d30>,
   <ipynb.fs.